## Load Data

In [2]:
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 12.9 MB/s eta 0:00:00


In [1]:
!pip install timm transformers pytorch-lightning Levenshtein rdkit wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 63.4 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.7/777.7 kB 80.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 51.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 117.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 108.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 120.1 MB/s eta 0:00:00


In [3]:
!pip install pytorch-lightning transformers -U

## Main

In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import timm
import torch
from torch import nn
from PIL import Image
from torch.utils.data import DataLoader, Dataset
import torch.backends.cudnn as cudnn
import wandb
import pytorch_lightning as pl
from torch.nn.utils.rnn import pad_sequence,pack_padded_sequence
from transformers import (AutoProcessor,
                          AutoTokenizer,
                          VisionEncoderDecoderModel,
                          RobertaTokenizerFast,
                          TrOCRForCausalLM,
                          AutoModel,
                          TrOCRConfig,
                          ViTModel,
                          ViTConfig,
                          ViTImageProcessor
                         )
from sklearn.model_selection import train_test_split
from rdkit import RDLogger,Chem
from rdkit.Chem import AllChem,DataStructs,MolFromSmiles,Draw
import torch.nn.functional as F
import os
from rdkit.DataStructs import TanimotoSimilarity
from Levenshtein import distance as levenshtein_distance
pl.seed_everything(56)

Seed set to 56


56

In [2]:
RDLogger.DisableLog('rdApp.*')
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
class CFG:
    wandb=False
    encoder="google/vit-base-patch16-384"
    decoder="entropy/roberta_zinc_480m"
    train_path = './MolScribe/train-00002-of-00003-1bb1afcf507d8618.parquet?download=true'#'./all_ChEMBLSmiles.csv'
    train_folder = './train/'
    betas=(0.9, 0.999)
    img_size = 512
    max_pred_len = 128
    val_split_size = 1e-6
    scheduler = None
    emb_dim = 512
    attention_dim = 512
    freq_threshold = 2
    decoder_dim = 512
    img_size=512
    dropout = 0.4
    eps=1e-6
    num_workers = 12
    batch_size = 192
    encoder_lr = 4e-4
    decoder_lr = 5e-4
    weight_decay = 0.01
    fine_tune_encoder = False
    max_epoches=6
    seed=56

## Data

In [4]:
class PLDataset(Dataset):
    def __init__(self, df, tokenizer,processor):
        super().__init__()
        self.df = df.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.processor = processor

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        label = self.df.iloc[idx]['smiles']
        image = self._gen_smiles(label)
        label_enc = self.tokenizer.encode_plus(label, padding='max_length',max_length=64, truncation=True, return_tensors='pt')
        return {'image':image,
                'input_ids':label_enc.input_ids.squeeze(0),
               'attention_mask':label_enc.attention_mask.squeeze(0)}
    
    def _gen_smiles(self,smiles):
        mol = MolFromSmiles(smiles)
        img = Draw.MolToImage(mol,size=(384,384))
        img = np.array(img.convert('L'))
        img = torch.from_numpy(img).unsqueeze(0) / 255
        return img

In [18]:
class PLDataModule(pl.LightningDataModule):
    def __init__(self,tokenizer,processor):
        super().__init__()
        self.cfg = CFG()
        self.is_setup = False
        self.tokenizer = tokenizer
        self.processor = processor

    def prepare_data(self):
        self.train_df = pd.read_parquet(CFG.train_path)[:1_000_000]

    def setup(self, stage: str):
        #self.train_df, self.val_df = train_test_split(self.train_data, test_size=self.cfg.val_split_size,random_state=self.cfg.seed)
        #self.train_df = self.train_df.reset_index(drop=True)
        #self.val_df = self.val_df.reset_index(drop=True)
        self.train_dataset = PLDataset(self.train_df,self.tokenizer,self.processor)
        #self.val_dataset = PLDataset(self.val_df,self.tokenizer,self.processor)
        self.is_setup = True

    def train_dataloader(self):
        return DataLoader(self.train_dataset,
                          batch_size=self.cfg.batch_size,
                          num_workers=self.cfg.num_workers,
                          pin_memory=True,
                          shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset,
                          batch_size=self.cfg.batch_size,
                          num_workers=self.cfg.num_workers,
                          pin_memory=True,
                          shuffle=False)

In [6]:
def char_accuracy(y_p,y):
    y_p,y = list(y_p),list(y)
    ln = min(len(y_p),len(y))
    score = 0
    for i in range(ln):
        if y_p[i] == y[i]:
            score += 1
    return score / max(len(y_p),len(y))

def correct_part(y_p):
    if Chem.MolFromSmiles(y_p) is None:
        return 0
    else:
        return 1

def tanimoto(y_p,y):
    try:
        mol1 = Chem.MolFromSmiles(y_p)
        mol2 = Chem.MolFromSmiles(y)

        vec_1 = AllChem.RDKFingerprint(mol1)
        vec_2 = AllChem.RDKFingerprint(mol2)
        return DataStructs.TanimotoSimilarity(vec_1,vec_2)
    except:
        return 0

In [7]:
class AverageMeter:
    def __init__(self):
        self.reset()

    def reset(self):
        self.preds = []
        self.targets = []

    def update(self,preds,targets):
        self.preds += preds
        self.targets += targets

    def calc_metrics(self):
        f = dict()

        f['char_acc'] = np.mean([char_accuracy(x,y) for x,y in zip(self.preds,self.targets)])
        f['corrent_part'] = np.mean([correct_part(x) for x in self.preds])
        f['tanimoto'] = np.mean([tanimoto(x,y) for x,y in zip(self.preds,self.targets)])

        return f

In [8]:
class PLModule(pl.LightningModule):
    def __init__(self,model,tokenizer):
        super().__init__()
        self.cfg = CFG()
        self.avg_meter = AverageMeter()
        self.model = model
        self.tokenizer = tokenizer

    def forward(self,image,input_ids=None,attention_mask=None):
        return self.model(pixel_values=image,labels=input_ids,decoder_attention_mask=attention_mask)

    def training_step(self, batch, _):
        loss = self(**batch).loss
        self.log_dict({'train_loss':loss.item()})
        return loss

    def validation_stepPAS(self, batch, _):
        labels = batch['input_ids'].detach().cpu().numpy()
        labels = [self.tokenizer.decode(ids, skip_special_tokens=True) for ids in labels]

        logits = self.model.generate(
            batch['image'],
            num_beams=4,
            max_length=128
        )

        logits = logits.detach().cpu().numpy()
        logits = [self.tokenizer.decode(ids, skip_special_tokens=True) for ids in logits]
        self.avg_meter.update(logits,labels)

    def predict_step(self,batch,_):

        logits = self.model.generate(
            batch['image'],
            num_beams=1,
            max_length=128
        )

        logits = logits.detach().cpu().numpy()
        logits = [self.tokenizer.decode(ids, skip_special_tokens=True) for ids in logits]
        return logits

    def on_validation_epoch_end(self):
        f = self.avg_meter.calc_metrics()
        self.log_dict(f)
        print(f)
        self.avg_meter.reset()

    def configure_optimizers(self):
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in self.model.encoder.named_parameters()],
                "lr":self.cfg.encoder_lr
            },
            {
                "params": [p for n, p in self.model.decoder.named_parameters()],
                "lr": self.cfg.decoder_lr
            },
        ]
        return torch.optim.AdamW(optimizer_grouped_parameters,
                                 betas=self.cfg.betas,
                                 weight_decay=self.cfg.weight_decay,
                                 eps=self.cfg.eps)

In [9]:
tokenizer = RobertaTokenizerFast.from_pretrained(CFG.decoder)
processor = AutoProcessor.from_pretrained(CFG.encoder)

vocab.json:   0%|          | 0.00/40.5k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/24.4k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

In [10]:
encoder = ViTModel(ViTConfig(hidden_size=384,
                             hidden_act='gelu',
                             image_size=384,
                             num_attention_heads=6,
                             num_hidden_layers=12,
                             num_channels=1,
                             intermediate_size=384 * 4,
                             patch_size=16))

In [11]:
decoder = TrOCRForCausalLM(TrOCRConfig(vocab_size=len(tokenizer),
                                       d_model=256,
                                       decoder_attention_heads=8,
                                       decoder_ffn_dim=1024,
                                       decoder_layers=6,
                                       activation_function='gelu',
                                       max_position_embeddings=384,
                                       dropout=0.2))

In [12]:
model = VisionEncoderDecoderModel(encoder=encoder,decoder=decoder)
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.vocab_size = model.config.decoder.vocab_size

In [13]:
dm = PLDataModule(tokenizer,processor)
dm.prepare_data()
dm.setup(0)

In [14]:
model_pl = PLModule(model,tokenizer)

In [15]:
wandb.login(key="673ae6e9b51cc896110db5327738b993795fffad")
os.environ['WANDB_API_KEY'] = "673ae6e9b51cc896110db5327738b993795fffad"
wandb.init(project='MOLECULA',name='TrOCR_tiny')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: Currently logged in as: andrey20007 (andrey2007). Use `wandb login --relogin` to force relogin


In [16]:
lr_monitor = pl.callbacks.LearningRateMonitor(logging_interval='epoch')
checkpoint_cb = pl.callbacks.ModelCheckpoint(
    dirpath='./outputs/',
    filename='base_model{epoch:02d}',
    monitor='tanimoto',
    mode='max',
    save_last=True
)

trainer = pl.Trainer(
    accelerator="gpu",
    precision=32,
    callbacks = [lr_monitor,checkpoint_cb],
    logger = pl.loggers.WandbLogger(),
    min_epochs=1,
    devices='auto',
    check_val_every_n_epoch=1,
    max_epochs=CFG.max_epoches
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [19]:
trainer.fit(model_pl,datamodule=dm)

/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/configuration_validator.py:72: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
You are using a CUDA device ('NVIDIA A100-SXM4-80GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/loggers/wandb.py:389: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:639: Checkpoint directory /notebooks/outputs exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVIC

Training: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.9/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:369: `ModelCheckpoint(monitor='tanimoto')` could not find the monitored key in the returned metrics: ['lr-AdamW/pg1', 'lr-AdamW/pg2', 'train_loss', 'epoch', 'step']. HINT: Did you call `log('tanimoto', value)` in the `LightningModule`?
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [21]:
torch.save(model_pl.state_dict(),'trocr_1m.pt')

[rank: 0] Received SIGTERM: 15
